In [ ]:
import os
import torch
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from Utils.niiparsing import load_nii
import glob

from Utils.argparsing import get_args
from Preprocessing.data_init import organized_data_download
from Preprocessing.data_loading import BratsDataset

if __name__ == "__main__":

    data = BratsDataset()

In [ ]:
rus = RandomUnderSampler(sampling_strategy=1, random_state=42)
x_res, y_res = rus.fit_resample(data.x, data.y)

zeros = 0
ones = 0

for val in y_res:
    if val==0:
        zeros+=1
    if val==1:
        ones+=1      

print(zeros)
print(ones)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x_res, y_res, test_size=0.6, random_state=42)
svc = SVC(random_state=42)
svc.fit(x_train, y_train)

In [ ]:
# testing on a single image

x_valid = []
y_valid = []

# Loading and normalizing whole brain images
x = load_nii('C:/Users/Jason/GitHub/Tumor-Segmentation/organized_data/HGG/BraTS19_2013_2_1_flair.nii.gz')
x = x / np.linalg.norm(x)
y = load_nii('C:/Users/Jason/GitHub/Tumor-Segmentation/organized_data/HGG/BraTS19_2013_2_1_seg.nii.gz')

# Padding
x_arr = np.pad(x, ((1, 1), (1, 1), (0, 0)), mode='constant')
y_arr = np.pad(y, ((1, 1), (1, 1), (0, 0)), mode='constant')

assert x_arr.shape[0] == x_arr.shape[1]

# For each 2D slice
for depth in range(x_arr.shape[2]):

    # Parameterizing sliding window
    for height in range(x_arr.shape[0] - 2) :
        for width in range(x_arr.shape[1] - 2):

            feature_vec = []

            # Creating sliding window across slice
            x_window = x_arr[height : height + 3, width : width + 3, depth]
            y_window = y_arr[height + 1, width + 1, depth]
                
            # Feature extraction
            feature_vec.append(np.mean(x_window))
            feature_vec.append(np.std(x_window))
            feature_vec.append(np.var(x_window))

            x_valid.append(feature_vec)

            if y_window == 0:
                y_valid.append(0)
            else:
                y_valid.append(1)
                
predictions = svc.predict(x_valid)
    
# calculate dice coefficient
y_test_trans = np.transpose(y_valid)
dice = (2*np.matmul(y_test_trans, predictions)) / (np.sum(y_valid + predictions))
dice_total += dice
print('Dice coefficient is %.6f' % dice)

In [ ]:
svc.score(x_valid, y_valid)

In [ ]:
# finding average dice coefficient of all images

count = 0
dice_total = 0

for filename in glob.glob('C:/Users/Jason/GitHub/Tumor-Segmentation/organized_data/*/*flair.nii.gz'):
    
    split = filename.split('flair')
    split[0] + 'seg' + split[1]

    x_valid = []
    y_valid = []

    # Loading and normalizing whole brain images
    x = load_nii(filename)
    x = x / np.linalg.norm(x)
    y = load_nii(split[0] + 'seg' + split[1])

    # Padding
    x_arr = np.pad(x, ((1, 1), (1, 1), (0, 0)), mode='constant')
    y_arr = np.pad(y, ((1, 1), (1, 1), (0, 0)), mode='constant')

    assert x_arr.shape[0] == x_arr.shape[1]

    # For each 2D slice
    for depth in range(x_arr.shape[2]):

        # Parameterizing sliding window
        for height, width in zip(range(x_arr.shape[0] - 2), range(x_arr.shape[1] - 2)):

            feature_vec = []

            # Creating sliding window across slice
            x_window = x_arr[height : height + 3, width : width + 3, depth]
            y_window = y_arr[height + 1, width + 1, depth]
                
            # Feature extraction
            feature_vec.append(np.mean(x_window))
            feature_vec.append(np.std(x_window))
            feature_vec.append(np.var(x_window))

            x_valid.append(feature_vec)

            if y_window == 0:
                y_valid.append(0)
            else:
                y_valid.append(1)
                
    count += 1
    predictions = svc.predict(x_valid)
    
    # calculate dice coefficient
    y_test_trans = np.transpose(y_valid)
    dice = (2*np.matmul(y_test_trans, predictions)) / (np.sum(y_valid + predictions))
    dice_total += dice
    print('Dice coefficient is %.6f' % dice)


In [ ]:
dice_avg = dice_total/count
print('Average dice coefficient is %.6f' % dice_avg)